In [ ]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# A simple Backtracking Constraint Solver

The module `extractVariables` implements the function `extractVars`.  This function takes one argument $f$ which is supposed to be a Python expression.  The function returns a set of all variables occuring in $f$. 

In [ ]:
import extractVariables as ev

In [ ]:
ev.extractVars("A == 𝜋 * r ** 2")

The input to the function `solve` is a *constraint satisfaction problem* `CSP`.  
The function `solve` tries to compute a solution of this problem via *backtracking*.
Its main purpose is to transform the given *CSP* into an *annotated CSP* where all the formulas 
are *annotated* with their variables, i.e. the third component of the *CSP* is now no longer a set of formulas but rather a set of pairs of the form `(f, V)` where `f` is a formula and `V`is the set of variables occurring in this formula.  It then calls `backtrack_search` to solve the *annotated CSP*.

In [ ]:
def solve(CSP):
    'Compute a solution for the given constraint satisfaction problem.'
    Variables, Values, Constraints = CSP
    CSP = (Variables,
           Values,
           [(f, ev.extractVars(f) & set(Variables)) for f in Constraints]
          )
    AllSolutions = []
    try:
        backtrack_search({}, CSP, AllSolutions)
    except Backtrack:
        return AllSolutions

Given a consistent *partial variable assignment* `Assignment` and a constraint satisfaction problem `CSP`,
the function`backtrack_search` tries to extend the given assignment recursively and thereby produce a solution of the given CSP.
All solutions that are found are added to the list `AllSolutions`.

In [ ]:
def backtrack_search(Assignment, CSP, AllSolutions):
    '''
    Given a partial variable assignment, this function tries to complete this 
    assignment towards a solution of the CSP.
    '''
    Variables, Values, Constraints = CSP
    if len(Assignment) == len(Variables):
        AllSolutions.append(Assignment)
        raise Backtrack()
    var = [x for x in Variables if x not in Assignment][0]
    for value in Values:
        try:
            if isConsistent(var, value, Assignment, Constraints):
                NewAssign      = Assignment.copy()
                NewAssign[var] = value
                backtrack_search(NewAssign, CSP, AllSolutions)
        except Backtrack:
            continue
    # all values have been tried 
    raise Backtrack()
    
class Backtrack(Exception):
    pass

The function `isConsistent` takes four arguments:
- `var` is a variable.
- `value` is a value that is to be assigned to the variable `var`.
- `Assignment` is a partial variable assignment that does not assign a value for `var`
  and that is consistent with all constraints in the set `Constraints`.
- `Constraints` is a set of logical formulas.

The function checks whether the assignment 
$$ \texttt{Assignment} \cup \{\texttt{var} \mapsto \texttt{value}\}$$
violates any of the formulas in `Constraints`.

In [ ]:
def isConsistent(var, value, Assignment, Constraints):
    NewAssign      = Assignment.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if var in Vs and Vs <= NewAssign.keys()
              )